<a href="https://colab.research.google.com/github/sisiflorensia/udacity-machine_learning_nanodegree/blob/master/Fine_tuning_Transformer_Models_Indobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installation & Load Dataset

In [1]:
!pip install transformers accelerate --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

import evaluate
import torch
import numpy as np
import pandas as pd
import json

# Load the dataset
file_path = 'november.json'
with open(file_path, 'r') as f:
    data = json.load(f)

# Convert JSON to DataFrame
df = pd.DataFrame(data)
print(df.head())

           sumber     tanggal        kategori  \
0  tribunnews.com  2015/11/01  Bisnis Ekonomi   
1  tribunnews.com  2015/11/01  Bisnis Ekonomi   
2  tribunnews.com  2015/11/01  Bisnis Ekonomi   
3  tribunnews.com  2015/11/01  Bisnis Ekonomi   
4  tribunnews.com  2015/11/01  Bisnis Ekonomi   

                                               judul  \
0  Anggota DPR: Kenapa BUMN Diberi Subsidi Rakyat...   
1  DPR Bingung Anggaran Subsidi Listrik Rp 38 Tri...   
2  Jatah Subsidi Listrik Berkurang Picu Kecemburu...   
3  Migrasi Penggunaan Daya Listrik Dinilai Bisa B...   
4  Citi Indonesia Merevolusi Pengalaman Perbankan...   

                                                 isi  \
0  TRIBUNNEWS.COM, JAKARTA - Anggota Komisi VII D...   
1  TRIBUNNEWS.COM, JAKARTA - Anggota Komisi VII D...   
2  TRIBUNNEWS.COM, JAKARTA - Pengamat Ekonomi dar...   
3  TRIBUNNEWS.COM, JAKARTA - Rencana pemerintah y...   
4  TRIBUNNEWS.COM, JAKARTA - Sebagai bentuk konsi...   

                               

# 2. Data Preprocessing

In [4]:
# Use the 'kategori' column to numerical labels using label2id
label_names = ['Bisnis Ekonomi', 'Teknologi', 'Travel', 'Otomotif', 'Olahraga', 'Nasional', 'Lifestyle', 'Bola']
label2id = {label: i for i, label in enumerate(label_names)}  # Create a mapping from label to id
df['label'] = df['kategori'].map(label2id)

# Only using 'judul' and 'isi' as input
df_selected = df[['judul','isi', 'label', 'kategori']]

# Check new dataframe content
print(df_selected.head())

                                               judul  \
0  Anggota DPR: Kenapa BUMN Diberi Subsidi Rakyat...   
1  DPR Bingung Anggaran Subsidi Listrik Rp 38 Tri...   
2  Jatah Subsidi Listrik Berkurang Picu Kecemburu...   
3  Migrasi Penggunaan Daya Listrik Dinilai Bisa B...   
4  Citi Indonesia Merevolusi Pengalaman Perbankan...   

                                                 isi  label        kategori  
0  TRIBUNNEWS.COM, JAKARTA - Anggota Komisi VII D...      0  Bisnis Ekonomi  
1  TRIBUNNEWS.COM, JAKARTA - Anggota Komisi VII D...      0  Bisnis Ekonomi  
2  TRIBUNNEWS.COM, JAKARTA - Pengamat Ekonomi dar...      0  Bisnis Ekonomi  
3  TRIBUNNEWS.COM, JAKARTA - Rencana pemerintah y...      0  Bisnis Ekonomi  
4  TRIBUNNEWS.COM, JAKARTA - Sebagai bentuk konsi...      0  Bisnis Ekonomi  


In [5]:
import re
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords

# Download NLTK data
nltk.download('punkt_tab')
nltk.download('stopwords')

indonesian_stopwords = set(stopwords.words('indonesian'))

# Text preprocessing: lowercasing and stopword removal
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    return text

df_selected['cleaned_judul'] = df_selected['judul'].apply(preprocess_text)
df_selected['cleaned_isi'] = df_selected['isi'].apply(preprocess_text)

df_selected

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-5-160983910df8>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['cleaned_judul'] = df_selected['judul'].apply(preprocess_text)
<ipython-input-5-160983910df8>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['cleaned_isi'] = df_selected['isi'].ap

,judul,isi,label,kategori,cleaned_judul,cleaned_isi
0,Anggota DPR: Kenapa BUMN Diberi Subsidi Rakyat...,"TRIBUNNEWS.COM, JAKARTA - Anggota Komisi VII D...",0,Bisnis Ekonomi,anggota dpr kenapa bumn diberi subsidi rakyat ...,tribunnewscom jakarta anggota komisi vii dpr ...
1,DPR Bingung Anggaran Subsidi Listrik Rp 38 Tri...,"TRIBUNNEWS.COM, JAKARTA - Anggota Komisi VII D...",0,Bisnis Ekonomi,dpr bingung anggaran subsidi listrik rp trili...,tribunnewscom jakarta anggota komisi vii dpr ...
2,Jatah Subsidi Listrik Berkurang Picu Kecemburu...,"TRIBUNNEWS.COM, JAKARTA - Pengamat Ekonomi dar...",0,Bisnis Ekonomi,jatah subsidi listrik berkurang picu kecemburu...,tribunnewscom jakarta pengamat ekonomi dari u...
3,Migrasi Penggunaan Daya Listrik Dinilai Bisa B...,"TRIBUNNEWS.COM, JAKARTA - Rencana pemerintah y...",0,Bisnis Ekonomi,migrasi penggunaan daya listrik dinilai bisa b...,tribunnewscom jakarta rencana pemerintah yang...
4,Citi Indonesia Merevolusi Pengalaman Perbankan...,"TRIBUNNEWS.COM, JAKARTA - Sebagai bentuk konsi...",0,Bisnis Ekonomi,citi indonesia merevolusi pengalaman perbankan...,tribunnewscom jakarta sebagai bentuk konsiste...
...,...,...,...,...,...,...
24187,Liverpool Gagal Menang Besar di Anfield,VIVA.co.id - Walau tampil mendominasi sejak ba...,7,Bola,liverpool gagal menang besar di anfield,vivacoid walau tampil mendominasi sejak babak...
24188,Arsenal Ditahan Imbang Norwich,VIVA.co.id - Arsenal gagal untuk membuntuti pe...,7,Bola,arsenal ditahan imbang norwich,vivacoid arsenal gagal untuk membuntuti perol...
24189,Bali United Ingin Patenkan Hansamu Yama dan Pa...,VIVA.co.id - Langkah Bali United di turnamen P...,7,Bola,bali united ingin patenkan hansamu yama dan pa...,vivacoid langkah bali united di turnamen pial...
24190,LIVE MATCH: Arsenal Hampir Dipermalukan Norwic...,VIVA.co.id - Arsenal mendominasi penguasaan bo...,7,Bola,live match arsenal hampir dipermalukan norwich...,vivacoid arsenal mendominasi penguasaan bola ...


In [6]:
# Split into training (70%), validation (10%), and test (20%) sets
x_train, x_train_temp, y_train, y_train_temp = train_test_split(
    df_selected[['cleaned_judul', 'cleaned_isi']], df_selected['label'], test_size=0.3, random_state=42)

x_val, x_test, y_val, y_test = train_test_split(
    x_train_temp, y_train_temp, test_size=2/3, random_state=42)

# Combine 'cleaned_judul' and 'cleaned_isi' into a single 'text' column for each split
x_train['text'] = x_train['cleaned_judul'] + ' ' + x_train['cleaned_isi']
x_val['text'] = x_val['cleaned_judul'] + ' ' + x_val['cleaned_isi']
x_test['text'] = x_test['cleaned_judul'] + ' ' + x_test['cleaned_isi']

# Now create your dataset using the 'text' column
dataset = DatasetDict({
    'train': Dataset.from_dict({'label': y_train.astype(int).tolist(), 'text': x_train['text'].tolist()}),
    'validation': Dataset.from_dict({'label': y_val.astype(int).tolist(), 'text': x_val['text'].tolist()}),
    'test': Dataset.from_dict({'label': y_test.astype(int).tolist(), 'text': x_test['text'].tolist()})
})

print(f"Training set size: {len(x_train)}")
print(f"Validation set size: {len(x_val)}")
print(f"Test set size: {len(x_test)}")

dataset

Training set size: 16934
Validation set size: 2419
Test set size: 4839


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 16934
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 2419
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 4839
    })
})

In [7]:
# Load IndoBERT model
model_checkpoint = 'indobenchmark/indobert-base-p2'
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=8)
# display architecture
model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
# create tokenize function
def tokenize_function(cleaned_dataset):
    # extract text
    text = cleaned_dataset["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [10]:
# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/16934 [00:00<?, ? examples/s]

Map:   0%|          | 0/2419 [00:00<?, ? examples/s]

Map:   0%|          | 0/4839 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16934
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2419
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4839
    })
})

# 3. Fine-tuning Process

We want to be able to quickly see model performance in categorising a couple body of text then compare the before and after fine tuning results on small data of text. Thus, we generate a small text sample and run the pre-trained model before we start the fine-tuning process.

In [11]:
# define list of examples
text_list = ["Musnah Sudah Mimpi Treble Inter", "Alhasil, Inter kalah agregat total 1-4 setelah berimbang 1-1 pada leg pertama. Hasil yang melanjutkan catatan buruk Inter beberapa hari belakangan ini.", "Kutukan Derby Inter Milan: Tanpa Kemenangan di 5 Duel Musim Ini", "Harga Emas Hari Ini Makin Ambruk, Jauhi Rp 2 Juta/Gram", "Rosan menjelaskan, industri pertambangan ini bergantung kepada harga komoditas dan operasional. Dalam hal harga komoditas, Rosan mengatakan hal ini merupakan faktor eksternal yang tidak bisa dikendalikan oleh pemerintah maupun pelaku industri."]

# Labels are 0-indexed and correspond to specific categories
label_names = ['Bisnis Ekonomi', 'Teknologi', 'Travel', 'Otomotif', 'Olahraga', 'Nasional', 'Lifestyle', 'Bola']
id2label = {i: label for i, label in enumerate(label_names)}

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
Musnah Sudah Mimpi Treble Inter - Bisnis Ekonomi
Alhasil, Inter kalah agregat total 1-4 setelah berimbang 1-1 pada leg pertama. Hasil yang melanjutkan catatan buruk Inter beberapa hari belakangan ini. - Lifestyle
Kutukan Derby Inter Milan: Tanpa Kemenangan di 5 Duel Musim Ini - Travel
Harga Emas Hari Ini Makin Ambruk, Jauhi Rp 2 Juta/Gram - Bola
Rosan menjelaskan, industri pertambangan ini bergantung kepada harga komoditas dan operasional. Dalam hal harga komoditas, Rosan mengatakan hal ini merupakan faktor eksternal yang tidak bisa dikendalikan oleh pemerintah maupun pelaku industri. - Lifestyle


In [12]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [13]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['attention.self.query'])

In [14]:
peft_config

LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=4, target_modules={'attention.self.query'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [15]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 79,880 || all params: 124,527,376 || trainable%: 0.0641


In [22]:
# hyperparameters random choices
lr = 2e-4
batch_size = 32
num_epochs = 8

In [23]:
# create data collator
# this will dynamically pad examples in each batch to be equal length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [25]:
# define an evaluation function to pass into trainer
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)
    }

In [26]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [27]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

# Saving the Fine-Tuned Model
save_directory = "./trained-indobert-for-news-2025"
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

<ipython-input-27-4bdc3a0d4f57>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sisiflorensia-ugm (sisiflorensia-ugm-universitas-gadjah-mada-library) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.564400,0.237035,{'accuracy': 0.9280694501860273}
2,0.195800,0.180298,{'accuracy': 0.9454319966928483}
3,0.151900,0.161092,{'accuracy': 0.9495659363373294}
4,0.127300,0.159772,{'accuracy': 0.9491525423728814}
5,0.117300,0.155827,{'accuracy': 0.954526663910707}
6,0.106900,0.156976,{'accuracy': 0.9508061182306738}
7,0.097600,0.163065,{'accuracy': 0.9536998759818107}
8,0.091900,0.158751,{'accuracy': 0.9536998759818107}


Trainer is attempting to log a value of "{'accuracy': 0.9280694501860273}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9454319966928483}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9495659363373294}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9491525423728814}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.954526663910707}" of

('./trained-indobert-for-news-2025/tokenizer_config.json',
 './trained-indobert-for-news-2025/special_tokens_map.json',
 './trained-indobert-for-news-2025/vocab.txt',
 './trained-indobert-for-news-2025/added_tokens.json',
 './trained-indobert-for-news-2025/tokenizer.json')

# 4. Evaluation

In [28]:
from sklearn.metrics import classification_report
from tqdm import tqdm  # Import tqdm for progress bar

device = torch.device("cpu")
model = model.to(device)

# Load the base model and tokenizer
base_model = AutoModelForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p2", num_labels=8)
base_tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")

# Load the fine-tuned model and tokenizer
finetuned_model = AutoModelForSequenceClassification.from_pretrained("./trained-indobert-for-news-2025", num_labels=8)
finetuned_tokenizer = AutoTokenizer.from_pretrained("./trained-indobert-for-news-2025")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# define list of examples
text_list = ["Musnah Sudah Mimpi Treble Inter", "Alhasil, Inter kalah agregat total 1-4 setelah berimbang 1-1 pada leg pertama. Hasil yang melanjutkan catatan buruk Inter beberapa hari belakangan ini.", "Kutukan Derby Inter Milan: Tanpa Kemenangan di 5 Duel Musim Ini", "Harga Emas Hari Ini Makin Ambruk, Jauhi Rp 2 Juta/Gram", "Rosan menjelaskan, industri pertambangan ini bergantung kepada harga komoditas dan operasional. Dalam hal harga komoditas, Rosan mengatakan hal ini merupakan faktor eksternal yang tidak bisa dikendalikan oleh pemerintah maupun pelaku industri."]

# Labels are 0-indexed and correspond to specific categories
label_names = ['Bisnis Ekonomi', 'Teknologi', 'Travel', 'Otomotif', 'Olahraga', 'Nasional', 'Lifestyle', 'Bola']
id2label = {i: label for i, label in enumerate(label_names)}

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
Musnah Sudah Mimpi Treble Inter - Olahraga
Alhasil, Inter kalah agregat total 1-4 setelah berimbang 1-1 pada leg pertama. Hasil yang melanjutkan catatan buruk Inter beberapa hari belakangan ini. - Olahraga
Kutukan Derby Inter Milan: Tanpa Kemenangan di 5 Duel Musim Ini - Bola
Harga Emas Hari Ini Makin Ambruk, Jauhi Rp 2 Juta/Gram - Bisnis Ekonomi
Rosan menjelaskan, industri pertambangan ini bergantung kepada harga komoditas dan operasional. Dalam hal harga komoditas, Rosan mengatakan hal ini merupakan faktor eksternal yang tidak bisa dikendalikan oleh pemerintah maupun pelaku industri. - Bisnis Ekonomi


In [34]:
def get_predictions(model, tokenizer, texts, batch_size=32):  # Reduced batch size
    model.eval()  # Set the model to evaluation mode
    device = model.device  # Get the device the model is on

    all_predictions = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Processing batches"):
        batch_texts = texts[i:i + batch_size]
        # Explicitly set max_length and truncation
        inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1).cpu().numpy()  # Move predictions to CPU
        all_predictions.extend(predictions)

    return np.array(all_predictions)

In [35]:
# Prepare test data
test_texts = dataset['test']['text']
test_labels = dataset['test']['label']
label_mapping = {'Bisnis Ekonomi': 0, 'Teknologi': 1, 'Travel': 2, 'Otomotif': 3, 'Olahraga': 4, 'Nasional': 5, 'Lifestyle': 6, 'Bola': 7}

In [36]:
# Get predictions for the base model
base_preds = get_predictions(base_model, base_tokenizer, test_texts)

# Get predictions for the fine-tuned model
finetuned_preds = get_predictions(finetuned_model, finetuned_tokenizer, test_texts)

Processing batches: 100%|██████████| 152/152 [17:06<00:00,  6.75s/it]


In [37]:
# Generate classification reports
print("=== Base Model Report ===")
print(classification_report(test_labels, base_preds, target_names=label_mapping))

print("\n=== Fine-Tuned Model Report ===")
print(classification_report(test_labels, finetuned_preds, target_names=label_mapping))

=== Base Model Report ===
                precision    recall  f1-score   support

Bisnis Ekonomi       0.00      0.00      0.00       773
     Teknologi       0.00      0.00      0.00       236
        Travel       0.00      0.00      0.00       149
      Otomotif       0.04      1.00      0.07       154
      Olahraga       0.00      0.00      0.00       274
      Nasional       0.70      0.13      0.22      2371
     Lifestyle       0.00      0.00      0.00       196
          Bola       0.09      0.00      0.00       686

      accuracy                           0.10      4839
     macro avg       0.10      0.14      0.04      4839
  weighted avg       0.36      0.10      0.11      4839


=== Fine-Tuned Model Report ===
                precision    recall  f1-score   support

Bisnis Ekonomi       0.92      0.90      0.91       773
     Teknologi       0.97      0.91      0.94       236
        Travel       0.93      0.93      0.93       149
      Otomotif       0.90      0.97      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [40]:
import shutil
import os
from google.colab import files

def zip_and_download(directory_path, zip_filename="trained-indobert-for-news-2025.zip"):
    """Zips a directory and downloads the archive."""
    if os.path.exists(directory_path):
        shutil.make_archive(directory_path, 'zip', directory_path)  # Create the archive
        files.download(zip_filename)  # Download it
    else:
        print(f"Error: Directory not found: {directory_path}")

# Example usage
directory_to_download = "./trained-indobert-for-news-2025"  # Replace with the directory you want to download
zip_and_download(directory_to_download)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>